In [33]:
import pandas as pd
import yfinance as yf
import psycopg2
from psycopg2 import sql
from sqlalchemy import create_engine, text
from psycopg2.errors import UniqueViolation

In [44]:
# Details for connection and api requests

# Database connection details
database = 'fin_data'
user = 'stevengiallourakis'
password = 'Birdman11!' # this might not be needed depenending on where you are connecting from.
host = 'localhost'
port =  '5433'
connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)

# Details for api request

symbol = 'LIN'
start_date = '2020-01-01'
end_date = '2024-07-04'

In [45]:
# Get Stock Data
def get_stock_data(symbol, start_date, end_date):
    stock_data = yf.download(symbol, start=start_date, end=end_date)
    stock_data = stock_data.drop(columns = ['High', 'Low', 'Adj Close', 'Volume'])
    stock_data['daily_change'] = stock_data['Close'] - stock_data['Open']
    stock_data['symbol'] = symbol
    stock_data.reset_index(inplace = True)
    print("Stock_data downloaded, High and Low columns removed, daily_change added")
    return stock_data 

# Add new data to database

def data_to_db(df, table_name, engine):
    try:
        with engine.begin() as connection:
            for index, row in df.iterrows():
                sql = text(f"""
                    INSERT INTO {table_name} ("Date", "Open", "Close", daily_change, symbol)
                    VALUES (:Date, :Open, :Close, :daily_change, :symbol)
                    ON CONFLICT (symbol, "Date") DO NOTHING
                """)
                connection.execute(sql, {
                    'Date': row['Date'],
                    'Open': row['Open'],
                    'Close': row['Close'],
                    'daily_change': row['daily_change'],
                    'symbol': row['symbol']
                })
        

        
        #df.to_sql(name=table_name, con=engine, if_exists='append', index=False)
        #print("Data successfully added to the database.")



    
    except UniqueViolation as e:
        print(f"Duplicate entry found for {symbol} on {data['date']}. Skipping insert.")
    except Exception as e:
        print(f"An error occurred while inserting data: {e}")
    

In [46]:
#Test

stock_data = get_stock_data(symbol, start_date, end_date)

data_to_db(stock_data, 'stocks', engine)

print(stock_data.head())

[*********************100%%**********************]  1 of 1 completed


Stock_data downloaded, High and Low columns removed, daily_change added
        Date        Open       Close  daily_change symbol
0 2020-01-02  213.580002  210.740005     -2.839996    LIN
1 2020-01-03  207.399994  205.259995     -2.139999    LIN
2 2020-01-06  204.600006  204.389999     -0.210007    LIN
3 2020-01-07  204.809998  204.830002      0.020004    LIN
4 2020-01-08  205.270004  207.389999      2.119995    LIN


In [60]:
#NASDAQ 100 A-C
nasdaq_100_A_C = ['ANSS','AAPL','AMAT','GEHC','ASML','TEAM','ADSK']

start_date = '2020-01-01'
end_date = '2024-07-04'


In [61]:
# Iterate through list of stocks

for symbol in nasdaq_100_A_C:
        print("Retrieving stock data")
            
        # Retrieve stock data and update dataframe to fit database
        stock_data = get_stock_data(symbol, start_date, end_date)


        # Check if the stock data is successfully downloaded
        if stock_data.empty:
            print(f"Failed to download data for {symbol}")
        else:
            print(f"Stock data downloaded and updated for {symbol}")

    
        # Send data to database
        data_to_db(stock_data, 'stocks', engine)



Retrieving stock data


[*********************100%%**********************]  1 of 1 completed


Stock_data downloaded, High and Low columns removed, daily_change added
Stock data downloaded and updated for ANSS
Retrieving stock data


[*********************100%%**********************]  1 of 1 completed


Stock_data downloaded, High and Low columns removed, daily_change added
Stock data downloaded and updated for AAPL
Retrieving stock data


...

Stock_data downloaded, High and Low columns removed, daily_change added
Stock data downloaded and updated for ASML
Retrieving stock data


[*********************100%%**********************]  1 of 1 completed


Stock_data downloaded, High and Low columns removed, daily_change added
Stock data downloaded and updated for TEAM
Retrieving stock data


[*********************100%%**********************]  1 of 1 completed


Stock_data downloaded, High and Low columns removed, daily_change added
Stock data downloaded and updated for ADSK
